In [1]:

import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier

codes are from here:
https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652

In [15]:

dataset = pd.read_csv('data/data.tsv', sep='\t')

cancerStatus = np.array(dataset.loc[:, dataset.columns == 'cancer'])

otus= np.array(dataset.loc[: , dataset.columns != 'cancer'])

# print (cancerStatus)

In [3]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_cancerStatus, test_cancerStatus,train_otus, test_otus  = train_test_split(cancerStatus, otus, test_size = 0.25, random_state = 42)

In [4]:
print('Training Features Shape:', train_otus.shape)
print('Training Labels Shape:', train_cancerStatus.shape)
print('Testing Features Shape:', test_otus.shape)
print('Testing Labels Shape:', test_cancerStatus.shape)

Training Features Shape: (219, 5206)
Training Labels Shape: (219, 1)
Testing Features Shape: (73, 5206)
Testing Labels Shape: (73, 1)


In [5]:
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()

rfc.fit(train_otus, train_cancerStatus.ravel())

rfc_predict = rfc.predict(test_otus)



In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc_cv_score = cross_val_score(rfc, otus, cancerStatus.ravel(), cv=10, scoring='roc_auc')

In [7]:
print("=== Confusion Matrix ===")
print(confusion_matrix(test_cancerStatus, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_cancerStatus, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[46  2]
 [12 13]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.79      0.96      0.87        48
           1       0.87      0.52      0.65        25

    accuracy                           0.81        73
   macro avg       0.83      0.74      0.76        73
weighted avg       0.82      0.81      0.79        73



=== All AUC Scores ===
[0.73611111 0.875      0.65441176 0.65931373 0.80882353 0.49264706
 0.73039216 0.86764706 0.66176471 0.81862745]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7304738562091504


In [8]:
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(train_otus, train_cancerStatus.ravel())
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END .max_depth=260, max_features=sqrt, n_estimators=800; total time=   4.6s
[CV] END .max_depth=260, max_features=sqrt, n_estimators=800; total time=   4.6s
[CV] END max_depth=140, max_features=auto, n_estimators=1000; total time=   5.8s
[CV] END max_depth=140, max_features=auto, n_estimators=1000; total time=   5.8s
[CV] END max_depth=140, max_features=auto, n_estimators=1000; total time=   5.8s
[CV] END max_depth=100, max_features=auto, n_estimators=1400; total time=   7.8s
[CV] END max_depth=100, max_features=auto, n_estimators=1400; total time=   7.8s
[CV] END max_depth=100, max_features=auto, n_estimators=1400; total time=   7.8s
[CV] END .max_depth=260, max_features=sqrt, n_estimators=800; total time=   4.1s
[CV] END .max_depth=420, max_features=sqrt, n_estimators=600; total time=   2.9s
[CV] END max_depth=300, max_features=auto, n_estimators=1000; total time=   5.0s
[CV] END .max_depth=420, max_features=sqrt, n_

In [9]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=220, max_features='auto')
rfc.fit(train_otus,train_cancerStatus.ravel())
rfc_predict = rfc.predict(test_otus)
rfc_cv_score = cross_val_score(rfc, otus, cancerStatus.ravel(), cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(test_cancerStatus.ravel(), rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_cancerStatus.ravel(), rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[46  2]
 [12 13]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.79      0.96      0.87        48
           1       0.87      0.52      0.65        25

    accuracy                           0.81        73
   macro avg       0.83      0.74      0.76        73
weighted avg       0.82      0.81      0.79        73



=== All AUC Scores ===
[0.78935185 0.8912037  0.6372549  0.7377451  0.78186275 0.52941176
 0.74509804 0.8872549  0.62745098 0.82843137]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7455065359477123


In [16]:
test= pd.read_csv('data/predata.tsv', sep='\t')

testotus= np.array(test.loc[: , test.columns != 'cancer'])

print(rfc.predict(testotus))




[0 0 1 1 1 0 1]
